## Qualitative change detection - vegetation changes in Redlands

Import library and search for Landsat imagery layer

In [ ]:
from arcgis.gis import GIS
gis = GIS('https://python.playground.esri.com/portal', 'arcgis_python', 'amazing_arcgis_123')
landsat_item = gis.content.search('Multispectral Landsat', 'Imagery Layer', outside_org=True)[0]
landsat = landsat_item.layers[0]
landsat_item

### Select images of Redlands area

In [ ]:
from arcgis.geocoding import geocode
area = geocode('Redlands, CA', out_sr=landsat.properties.spatialReference)[0]

In [ ]:
import arcgis
selected = landsat.filter_by(where="CloudCover <=0.10", 
                   geometry=arcgis.geometry.filters.intersects(area['extent']))

You can query the filtered rasters as a FeatureSet:

In [ ]:
fs = selected.query(out_fields="AcquisitionDate, CloudCover, WRS_Row, WRS_Path", 
              return_geometry=True,
              return_distinct_values=False,
              order_by_fields="AcquisitionDate")

Attributes of the selected rasters can be queried using a Pandas dataframe using the 'df' property of the FeatureSet.

In [ ]:
import pandas as pd
df = fs.df
df['Time'] = pd.to_datetime(df['AcquisitionDate'], unit='ms')
df[(df['Time'] > '2015-07') & (df['Time'] <'2016-05')]

### Change Detection

In [ ]:
summer_2015 = landsat.filter_by('OBJECTID=160262')
spring_2016 = landsat.filter_by('OBJECTID=278577')

#### NDVI diff

In [ ]:
ndvi_diff = ndvi(spring_2016, '5 4') - ndvi(summer_2015, '5 4')

#### Remap NDVI and apply a colormap

In [ ]:
threshold = 0.1
ndvi_diff_remap = remap(ndvi_diff, input_ranges=[threshold, 1], output_values=[1],
                       no_data_ranges=[-1, threshold], astype='u8')

ndvi_diff_colormap = colormap(ndvi_diff_remap, colormap=[[1, 124, 252, 0]], astype='u8')

The difference image and threshold mask can be visualized using the map widget:

In [ ]:
redlands_map = gis.map('Redlands, CA')
redlands_map.basemap='satellite'
redlands_map

In [ ]:
redlands_map.add_layer(ndvi_diff_colormap)